<a href="https://colab.research.google.com/github/SeongwonTak/TIL_swtak/blob/master/210120_Linear_Regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 210120 _ 선형회귀분석 1편
이전 플젝에서는 결과를 얻기 위해 모델을 찾아서 썼다면, 선형 회귀 분석 내용을 조금 깊게, 제대로 며칠에 걸쳐서 제대로 공부하고자 한다.
다룰 토픽이 상당히 많기에, 차근차근 공부할 것이다.

오늘은 데이터사이언스 스쿨 자료를 메인으로 하여 정리할 것이다.


## 1. 회귀 문제란?
특정한 입력 변수 값을 바탕으로 출력 값을 예측하는 문제를 생각하자.
이 중 출력변수가 연속형 변수일 경우, regression problem 이라고 한다.

앞으로는, Linear Regression의 case에 대해서만 다루려고 한다.

조금 더 구체적으로 알아보자, 

독립변수 $x$와 이에 대응되는 종속변수 $y$가 주어질 때,
$y$와 '가장 비슷한 값'  $\hat{y}$를 출력하는 함수 $f(x)$를 찾고자 한다.

이를 찾기 위해 선형함수를 사용할 경우, Linear Regression을 하게 된 것이라고 보면 된다.

여기서 궁금한 것은 **'가장 비슷한 값'**이란 무엇인가? 이다.

가장 비슷한 값은 단순히 하나의 변수에 대한 정확한 예측이 아닌, 전체적인 정확도를 판단해야 할 것이다. 이를 고려할 수 있는 변수는 분산 계산과 유사한 방식으로, **오차 제곱합(잔차 제곱합이라고도 한다, Residual Sum of Squares)을 최소화 해야 할 것**이다. (단순히, 편차의 합은 0이므로, 제곱합을 해야 0이 나오지 않는다!)

## 2. 회귀 문제 풀기
우리는 단순히 변수가 1개가 아닌 **다항선형회귀 경우**에 대해서 증명을 하려고 한다.


### 1. 상수항 결합
이것을 고민하는 이유는 간단하다.
선형회귀식을 자세히 들여다보면

$$ \hat{y}=w_{0}+w_{1}x_{1}+...+w_{n}x_{n}$$

이를 행렬의 곱셈 표현으로 하려고 하다보니 $x$ 벡터의 앞에 1이 들어가면 행렬의 곱셈 표현으로 깔끔하게 표현이 된다.
이렇게 상수항 1을 독립변수에 추가하는 것을 **상수항 결합**(bias augmentation)이라고 한다.

In [1]:
import numpy as np
import statsmodels.api as sm

X0 = np.arange(6).reshape(2, 3)
X = sm.add_constant(X0)  #bias_augmentation
X

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


array([[1., 0., 1., 2.],
       [1., 3., 4., 5.]])

### 2. 최소자승법

위의 과정을 거치면 
$$ \hat{y} = Xw $$
를 얻게 된다.

이 때, residual vector $ e $를 구하면,
$$ e = y - Xw $$

우리가 구해야 할 것은 RSS를 최소하 하는 것이다.
$$ RSS = e^{T}e = (y-Xw)^{T}(y-Xw) $$

$w$가 변함에 따른 RSS의 최솟값을 구해야 한다.
이것이 되는 $W$는, 

$$ \frac{dRSS}{dw} = 0$$

이 되는 지점(들) 중에서 존재할 것이다. 유일하다면 최대/최소를 확인해야 할 것이다.

이를 계산하여 정리하면,
$$ w = (X^{T}X)^{-1}X^{T}y$$
일 때, RSS가 최소화가 됨을 알 수 있다.

단, 조건은

- $X^{T}X$는 당연히, 역행렬이 존재해야 한다.
- $X^{T}X$가 positive definite여야 한다.

## 3. 코드 예시

결과를 알았으므로, 실제 예시를 통해 어떤 점들을 고려해야 하는지를 자세히 보려고 한다.

사용할 데이터는 boston 집값 데이터이다. (역시 유명한 그 예제)

먼저 statmodels를 활용한 방법이다.

In [3]:
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()

dfX0 = pd.DataFrame(boston.data, columns=boston.feature_names)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])

model_boston2 = sm.OLS(dfy, dfX)
result_boston2 = model_boston2.fit()
print(result_boston2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Wed, 20 Jan 2021   Prob (F-statistic):          6.72e-135
Time:                        14:09:53   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         36.4595      5.103      7.144      0.0

다음은 sklearn을 활용한 방법이다.
이 방법을 통해서는 특정 데이터가 주어질 때 결과값을 계산하기 편리해보인다.

##4. 모델의 해석
위의 statmodel은 회귀분석의 결과를 나타내는 결과표를 주기에 이에 대한 해석이 필요하다. 이에 대한 몇가지 의문점을 정리히고자 한다.

1. 모델의 정확도는 어떻게 알 수 있을까?
2. 계수는 어디서 읽어낼 수 있을까? - coef를 보면 된다.
3. 모든 계수가 다 의미가 있을까?
4. 어떤 변수의 범위는 너무 크고, 어떤 변수는 너무 작고, 어디는 몰려있을건데, 그대로 회귀분석을 해도 되나?
5. 대체, Cond NO.는 무엇이며, 저 밑의 경고는 왜 뜨는 걸까?
6. 모델을 쓰기 위한 조건은 더 없을까?

하나씩 확인해 보자.

### 1. 모델의 정확도 : R-squared vs Adjust R-squared

**결정계수라고 불리는 R-Squared** 란 회귀분석으로 만든 모형이 실제 데이터를 얼마만큼 잘 예측했는지를 나타내는 측도이다. 반응변수 Y의 전체 변이 중에서 예측변수들에 의하여 설명되는 비율이다.

- SSE (Sum of Squared Error) : $\sum_{i=1}^{n}(y_{i}-\hat{y}_{i})^{2}$
- SST (Total sum of squares) : $\sum_{i=1}^{n}(y_{i}-\mu_{y})^{2}$

이를 바탕으로 **'결정계수'란, 타깃의 분산을 모델이 얼마나 잡아냈는가?**로 요약할 수 있다.
$$ R^{2} = 1 - \frac{SSE}{SST}$$


하지만 이 값의 경우, 변수가 추가될수록 R값이 증가할 수 밖에 없다는 특성이 있어 실제 유의미한 변수를 반영하는 것을 중시하는 Adjust R-squared를 활용하게 된다.


### 2. 변수의 스케일링? 조건수?

먼저 조건수랑, 행렬에서 가장 큰 eigenvalue와 가장 작은 eigenvalue의 비율이다.
조건수가 커질수록, 회귀분석에서는 오차가 커지는 성향을 가지게 된다.

조건수를 줄이는 방법은 크게 두 가지를 볼 수 있다.
- 상관관계가 높은 변수들끼리 묶어 축약시킨다
- 변수들간의 범위 차이가 클 경우 scaling을 통해 범위를 조절한다.
(예시 - 표준화, min-max scaling, 로그변환)


회귀분석의 계수들이 의미를 가질 수 있는지, 다른 조건은 없는지, 그리고 범주형 데이터가 들어올 때는 어떻게 대응을 해야 하는지 등에 대해서도 알아봐야 할 것이다. 이는 선형회귀분석 2편에서 정리하고자 한다.

